In [14]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib import cbook

warnings.filterwarnings("ignore")
%matplotlib inline

task_data = "income"

In [15]:
# scores loading
import numpy as np
import pandas as pd

DATA_PATH = "../../src/data/evaluation"
TEST_PATH = f"../../src/data/acs_{task_data}/processed/acs_{task_data}_test.csv"

BASELINE = f"{DATA_PATH}/baseline/{task_data}"
SEPARATION = f"{DATA_PATH}/hardt2016/{task_data}"
INDENPENDENCE = f"{DATA_PATH}/kamiran_calders2012/{task_data}"
SUFFICIENCY = f"{DATA_PATH}/pleiss2017/{task_data}/calib_weighted"

base_pred = pd.read_csv(f"{BASELINE}/XGBClassifier_predictions.csv")
sep_pred = pd.read_csv(f"{SEPARATION}/XGBClassifier_separation_predictions.csv")
ind_pred = pd.read_csv(f"{INDENPENDENCE}/XGBClassifier_independence_predictions.csv")
suf_pred = pd.read_csv(f"{SUFFICIENCY}/XGBClassifier_sufficiency_predictions.csv")

base_scores = np.load(f"{BASELINE}/XGBClassifier_scores.npy", allow_pickle=True).item()
base_scores_cond = np.load(f"{BASELINE}/XGBClassifier_conditional_scores.npy", allow_pickle=True).item()

sep_scores = np.load(f"{SEPARATION}/XGBClassifier_scores_separation.npy", allow_pickle=True).item()
sep_scores_cond = np.load(f"{SEPARATION}/XGBClassifier_conditional_scores_separation.npy", allow_pickle=True).item()

ind_scores = np.load(f"{INDENPENDENCE}/XGBClassifier_scores_independence.npy", allow_pickle=True).item()
ind_scores_cond = np.load(
    f"{INDENPENDENCE}/XGBClassifier_conditional_scores_independence.npy", allow_pickle=True
).item()

suf_scores = np.load(f"{SUFFICIENCY}/XGBClassifier_scores_sufficiency.npy", allow_pickle=True).item()
suf_scores_cond = np.load(f"{SUFFICIENCY}/XGBClassifier_conditional_scores_sufficiency.npy", allow_pickle=True).item()

df_test = pd.read_csv(TEST_PATH)

In [16]:
# data loading as dataframes

df_base = pd.DataFrame.from_dict(base_scores, orient="index")
df_base_cond = pd.DataFrame.from_dict(base_scores_cond, orient="index")

df_suf = pd.DataFrame.from_dict(suf_scores, orient="index")
df_suf_cond = pd.DataFrame.from_dict(suf_scores_cond, orient="index")

In [ ]:
mpl.rcParams["figure.dpi"] = 100
colors = plt.get_cmap("Dark2")

box_colors = plt.get_cmap("Set3")
box_colors

In [ ]:
df_base_cond["PRIV_PPV"]

In [19]:
# If we are most worried about being unfair to people without assistance (rejected loan applicants),
# the best fairness metric would be something called False Omission Rate (FOR) parity.
# (Since FOR = 1-NPV, this metric also relates to the Sufficiency criterion.)

# Looking at FOR values, we see that 43% of women the model predicted to default actually would have
# paid back their loan – exactly the same proportion as among men. Even though the model falsely omits
# (or wrongly denies) 43 percent of female and male loan applicants from actually getting loans, it
# treats men and women the same. Therefore, in this case, if we focus on FOR as our fairness metric
# and females as our protected demographic group, we can confirm that our ML assisted credit scoring
# decisions do not show “any prejudice or favoritism toward an individual or group”. Fairness can be
# measured – once the right fairness definition to use has been identified by us.

# requires that the positive and negative predictive values are the same. Values such as the true
# positive rate and the positive predictive rate are easy to calculate for different populations,
# and they can be simply read off a confusion matrix.

In [ ]:
df_base_cond["UNP_PPV"].mean()

In [ ]:
labels = ["baseline", "calibration \n info_withholding"]
legend_labels = ["Females", "Males"]


ax1_data_lists = [
    (df_base_cond["UNP_PPV"].values, df_base_cond["PRIV_PPV"].values),
    (df_suf_cond["UNP_PPV"].values, df_suf_cond["PRIV_PPV"].values),
]

print("PPV means baseline:", df_base_cond["UNP_PPV"].mean(), df_base_cond["PRIV_PPV"].mean())
print("PPV means sufficiency:", df_suf_cond["UNP_PPV"].mean(), df_suf_cond["PRIV_PPV"].mean())

ax2_data_lists = [
    (df_base_cond["UNP_NPV"].values, df_base_cond["PRIV_NPV"].values),
    (df_suf_cond["UNP_NPV"].values, df_suf_cond["PRIV_NPV"].values),
]

print("NPV means baseline:", df_base_cond["UNP_NPV"].mean(), df_base_cond["PRIV_NPV"].mean())
print("NPV means sufficiency:", df_suf_cond["UNP_NPV"].mean(), df_suf_cond["PRIV_NPV"].mean())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.3))
# fig, ax1 = plt.subplots(figsize=(5, 4.5))

num_groups = len(ax1_data_lists)
group_width = 1
box_width = group_width / 4
positions = np.arange(num_groups)

box_properties = {
    "patch_artist": True,
    "showfliers": False,
    "medianprops": {"color": "black"},
    "whiskerprops": {"color": "black"},
    "capprops": {"color": "black"},
    "flierprops": {"markeredgecolor": "black"},
    "showmeans": True,
    "meanprops": {"marker": "o", "markerfacecolor": "white", "markeredgecolor": "black", "markersize": 4},
}

for i, (data1, data2) in enumerate(ax1_data_lists):

    # generate stats for the boxplot
    stats1 = cbook.boxplot_stats([data1], labels=[labels[0]])
    stats2 = cbook.boxplot_stats([data2], labels=[labels[1]])

    # Plot the boxplot statistics using bxp
    bp1 = ax1.bxp(stats1, positions=[positions[i] - box_width / 2], widths=box_width, **box_properties)
    bp2 = ax1.bxp(stats2, positions=[positions[i] + box_width / 2], widths=box_width, **box_properties)

    for patch in bp1["boxes"]:
        patch.set_facecolor(box_colors(3))
    for patch in bp2["boxes"]:
        patch.set_facecolor(box_colors(4))

    # get the limit of the y-axis
    y_max = max(data1.max(), data2.max())
    y_max = y_max + 0.0015
    # Adding the significance bar
    x1 = positions[i] - box_width / 2 - 0.02
    x2 = positions[i] + box_width / 2 + 0.02
    y, h, color = y_max, 0.001, "#3D3D3D"  # Adjust these values based on your plot scale

    # Plot the significance bar
    ax1.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.0, c=color)
    # ax2.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")
    if i == 0:
        ax1.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")
    else:
        ax1.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")

    # Add separator line between pairs, except after the last pair
    if i < num_groups - 1:
        ax1.axvline(x=positions[i] + 0.5, color="gray", linestyle="--", alpha=0.7)

# ax1.set_yticks([0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.21])
# ax1.set_ylim(0.08, 0.21)
ax1.set_xticks(positions)
ax1.set_xticklabels(labels)
ax1.yaxis.grid(True)
ax1.set_ylabel("error rates scale")
ax1.set_title("positive predictive values across groups", fontsize=10)


for i, (data1, data2) in enumerate(ax2_data_lists):
    # generate stats for the boxplot
    stats1 = cbook.boxplot_stats([data1], labels=[labels[0]])
    stats2 = cbook.boxplot_stats([data2], labels=[labels[1]])

    # Plot the boxplot statistics using bxp
    bp1 = ax2.bxp(stats1, positions=[positions[i] - box_width / 2], widths=box_width, **box_properties)
    bp2 = ax2.bxp(stats2, positions=[positions[i] + box_width / 2], widths=box_width, **box_properties)

    for patch in bp1["boxes"]:
        patch.set_facecolor(box_colors(3))
    for patch in bp2["boxes"]:
        patch.set_facecolor(box_colors(4))

    # get the limit of the y-axis
    y_max = max(data1.max(), data2.max())
    y_max = y_max + 0.0015
    # Adding the significance bar
    x1 = positions[i] - box_width / 2 - 0.02
    x2 = positions[i] + box_width / 2 + 0.02
    y, h, color = y_max, 0.001, "#3D3D3D"  # Adjust these values based on your plot scale

    # Plot the significance bar
    ax2.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.0, c=color)
    # ax2.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")
    if i == 0:
        ax2.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")
    else:
        ax2.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")

    if i < num_groups - 1:
        ax2.axvline(x=positions[i] + 0.5, color="gray", linestyle="--", alpha=0.7)

ax2.set_xticks(positions)
ax2.set_xticklabels(labels)
ax2.yaxis.grid(True)
ax2.set_ylabel("error rates scale")
ax2.set_title("negative predictive values across groups", fontsize=10)

first_legend = ax2.legend(labels=legend_labels, bbox_to_anchor=(1.04, 1), title="sensitive groups", loc="upper left")

from matplotlib.lines import Line2D

legend_elements = [
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="* p < 0.05"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="** p < 0.01"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="*** p < 0.001"),
]

# Adding the custom legend to the plot
# Add the second legend to the same axis, but outside of the plot
second_legend = ax2.legend(
    handles=legend_elements,
    loc="upper left",
    bbox_to_anchor=(1.05, 0.75),
    ncol=1,
    title="significance levels",
    fontsize=8,
)
ax2.add_artist(first_legend)

plt.tight_layout()
# plt.savefig("../assets/boxplot_ppv.png", dpi=300, bbox_inches="tight")
# plt.savefig("../assets/boxplot_fdr.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
labels = ["baseline", "calibration \n info_withholding"]
legend_labels = ["Females", "Males"]

ax1_data_lists = [
    (df_base_cond["UNP_FOR"].values, df_base_cond["PRIV_FOR"].values),
    (df_suf_cond["UNP_FOR"].values, df_suf_cond["PRIV_FOR"].values),
]

# If we are most worried about being unfair to people without assistance (rejected loan applicants),
# the best fairness metric would be something called False Omission Rate (FOR) parity.
# (Since FOR = 1-NPV, this metric also relates to the Sufficiency criterion.)

# Looking at FOR values, we see that 43% of women the model predicted to default actually would have
# paid back their loan – exactly the same proportion as among men. Even though the model falsely omits
# (or wrongly denies) 43 percent of female and male loan applicants from actually getting loans, it
# treats men and women the same. Therefore, in this case, if we focus on FOR as our fairness metric
# and females as our protected demographic group, we can confirm that our ML assisted credit scoring
# decisions do not show “any prejudice or favoritism toward an individual or group”. Fairness can be
# measured – once the right fairness definition to use has been identified by us.

ax2_data_lists = [
    (df_base_cond["UNP_FDR"].values, df_base_cond["PRIV_FDR"].values),
    (df_suf_cond["UNP_FDR"].values, df_suf_cond["PRIV_FDR"].values),
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4.3))
# fig, ax1 = plt.subplots(figsize=(5, 4.5))

num_groups = len(ax1_data_lists)
group_width = 1
box_width = group_width / 4
positions = np.arange(num_groups)

box_properties = {
    "patch_artist": True,
    "showfliers": False,
    "medianprops": {"color": "black"},
    "whiskerprops": {"color": "black"},
    "capprops": {"color": "black"},
    "flierprops": {"markeredgecolor": "black"},
}

for i, (data1, data2) in enumerate(ax1_data_lists):

    # generate stats for the boxplot
    stats1 = cbook.boxplot_stats([data1], labels=[labels[0]])
    stats2 = cbook.boxplot_stats([data2], labels=[labels[1]])

    # Plot the boxplot statistics using bxp
    bp1 = ax1.bxp(stats1, positions=[positions[i] - box_width / 2], widths=box_width, **box_properties)
    bp2 = ax1.bxp(stats2, positions=[positions[i] + box_width / 2], widths=box_width, **box_properties)

    for patch in bp1["boxes"]:
        patch.set_facecolor(box_colors(3))
    for patch in bp2["boxes"]:
        patch.set_facecolor(box_colors(4))

    # get the limit of the y-axis
    y_max = max(data1.max(), data2.max())
    y_max = y_max + 0.0015
    # Adding the significance bar
    x1 = positions[i] - box_width / 2 - 0.02
    x2 = positions[i] + box_width / 2 + 0.02
    y, h, color = y_max, 0.001, "#3D3D3D"  # Adjust these values based on your plot scale

    # Plot the significance bar
    ax1.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.0, c=color)
    # ax2.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")
    if i == 0:
        ax1.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")
    else:
        ax1.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")

    # Add separator line between pairs, except after the last pair
    if i < num_groups - 1:
        ax1.axvline(x=positions[i] + 0.5, color="gray", linestyle="--", alpha=0.7)

# ax1.set_yticks([0.10, 0.12, 0.14, 0.16, 0.18, 0.20, 0.21])
# ax1.set_ylim(0.08, 0.21)
ax1.set_xticks(positions)
ax1.set_xticklabels(labels)
ax1.yaxis.grid(True)
ax1.set_ylabel("error rates scale")
ax1.set_title("false omission rates across groups", fontsize=10)


for i, (data1, data2) in enumerate(ax2_data_lists):
    # generate stats for the boxplot
    stats1 = cbook.boxplot_stats([data1], labels=[labels[0]])
    stats2 = cbook.boxplot_stats([data2], labels=[labels[1]])

    # Plot the boxplot statistics using bxp
    bp1 = ax2.bxp(stats1, positions=[positions[i] - box_width / 2], widths=box_width, **box_properties)
    bp2 = ax2.bxp(stats2, positions=[positions[i] + box_width / 2], widths=box_width, **box_properties)

    for patch in bp1["boxes"]:
        patch.set_facecolor(box_colors(3))
    for patch in bp2["boxes"]:
        patch.set_facecolor(box_colors(4))

    # get the limit of the y-axis
    y_max = max(data1.max(), data2.max())
    y_max = y_max + 0.0015
    # Adding the significance bar
    x1 = positions[i] - box_width / 2 - 0.02
    x2 = positions[i] + box_width / 2 + 0.02
    y, h, color = y_max, 0.001, "#3D3D3D"  # Adjust these values based on your plot scale

    # Plot the significance bar
    ax2.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.0, c=color)
    # ax2.text((x1 + x2) * 0.5, y + h, "**", ha="center", va="bottom", color="black")
    if i == 0:
        ax2.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")
    else:
        ax2.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")

    if i < num_groups - 1:
        ax2.axvline(x=positions[i] + 0.5, color="gray", linestyle="--", alpha=0.7)

ax2.set_xticks(positions)
ax2.set_xticklabels(labels)
ax2.yaxis.grid(True)
ax2.set_ylabel("error rates scale")
ax2.set_title("false discovery rates across groups", fontsize=10)

first_legend = ax2.legend(labels=legend_labels, bbox_to_anchor=(1.04, 1), title="sensitive groups", loc="upper left")

from matplotlib.lines import Line2D

legend_elements = [
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="* p < 0.05"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="** p < 0.01"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="*** p < 0.001"),
]

# Adding the custom legend to the plot
# Add the second legend to the same axis, but outside of the plot
second_legend = ax2.legend(
    handles=legend_elements,
    loc="upper left",
    bbox_to_anchor=(1.05, 0.75),
    ncol=1,
    title="significance levels",
    fontsize=8,
)
ax2.add_artist(first_legend)

plt.tight_layout()
# plt.savefig("../assets/boxplot_ppv.png", dpi=300, bbox_inches="tight")
# plt.savefig("../assets/boxplot_fdr.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
df_base.columns

In [ ]:
mpl.rcParams["figure.dpi"] = 100
colors = plt.get_cmap("Dark2")

labels = ["baseline", "calibration \n info_withholding"]
suf_pred_values = [df_base["AVG_PRED_VALUE_DIFF"].values, df_suf["AVG_PRED_VALUE_DIFF"].values]

fig, ax = plt.subplots(figsize=(5, 4.5))

num_groups = len(suf_pred_values)
group_width = 1
box_width = group_width / 3
positions = np.arange(num_groups)

box_properties = {
    "patch_artist": True,
    "showfliers": False,
    "medianprops": {"color": "black"},
    "whiskerprops": {"color": "black"},
    "capprops": {"color": "black"},
    "flierprops": {"markeredgecolor": "black"},
}

print(suf_pred_values[0], suf_pred_values[1])
# independence
stats1 = cbook.boxplot_stats([suf_pred_values[0]], labels=["base"])
stats2 = cbook.boxplot_stats([suf_pred_values[1]], labels=["suf"])
bp1 = ax.bxp(stats1, positions=[positions[i] - box_width / 2], widths=box_width, **box_properties)
bp2 = ax.bxp(stats2, positions=[positions[i] + box_width / 2], widths=box_width, **box_properties)

for patch in bp1["boxes"]:
    patch.set_facecolor(colors(0))
for patch in bp2["boxes"]:
    patch.set_facecolor(colors(2))

# ax.set_xticks([])
ax.yaxis.grid(True)
# ax.set_xlabel("reweighting")
ax.set_ylabel("error rates scale")
ax.set_title("average predictive values difference", fontsize=10)

# Adding the significance bar
x1 = positions[i] - box_width / 2 - 0.02
x2 = positions[i] + box_width / 2 + 0.02

# # Plot the significance bar
# y_max = max(suf_pred_values[0].max(), suf_pred_values[1].max())
# y_max = y_max + 0.003
# y, h, color = y_max, 0.003, "#3D3D3D"  # Adjust these values based on your plot scale
# ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.0, c=color)
# ax.text((x1 + x2) * 0.5, y + h, "***", ha="center", va="bottom", color="black")

# Adding the custom legend to the plot
first_legend = ax.legend(labels=labels, bbox_to_anchor=(1.04, 1), title="intervention", loc="upper left")

text_properties = {
    "horizontalalignment": "right",
    "verticalalignment": "top",
    "color": "red",
    # "fontweight": "bold",
}
# Add annotations
ax.text(-0.01, 1.02, "fair", transform=ax.transAxes, **text_properties)
ax.text(-0.01, 0.05, "unfair", transform=ax.transAxes, **text_properties)

from matplotlib.lines import Line2D

legend_elements = [
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="* p < 0.05"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="** p < 0.01"),
    Line2D([0], [0], marker="None", color="none", lw=1.0, label="*** p < 0.001"),
]

# Add the second legend to the same axis, but outside of the plot
second_legend = ax.legend(
    handles=legend_elements,
    loc="upper left",
    bbox_to_anchor=(1.05, 0.75),
    ncol=1,
    title="significance levels",
    fontsize=8,
)
ax.add_artist(first_legend)
# ax.add_artist(second_legend)

plt.tight_layout()
# plt.savefig("../assets/boxplot_spd.png", dpi=300, bbox_inches="tight")
plt.show()